In [1]:
import numpy as np
from numpy.random import sample
import pandas as pd
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from imblearn.over_sampling import SMOTE
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import brier_score_loss, balanced_accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_curve, auc, accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.utils import resample
import matplotlib.pyplot as plt
%matplotlib inline

B = 501

## SCI

In [3]:
df = pd.read_csv("SCI.csv")
df = df.drop('StudyID', axis=1)
df = df.rename(columns={'FU_SA': 'Outcome'})
y = df.Outcome
X = df.drop('Outcome', axis=1)

## ND

df = pd.read_csv("nd.csv", encoding = "ISO-8859-1")

df['Outcome'] = df['3Actual _Attempt'] + df['3Interrupted_ attempt'] + df['3Aborted _Attempt']
df = df[df['Outcome'].notna()]
df = df.drop(['Study Id', 'age', 'annual income', 'marital stat', 'lives with', 'gender', 'race', 'ethnicity', 'yrs of edu', '3CSSRS-_most severe_ ideation', '3Actual _Attempt', '3Interrupted_ attempt', '3Aborted _Attempt'], axis=1)
df['Outcome'] = df['Outcome'].astype(bool).astype(int)
df=df.dropna(axis=0,how='any')
df=df.dropna(axis=1,how='any')
#print(df['Outcome'].value_counts())

y = df.Outcome
X = df.drop('Outcome', axis=1)

## Garbage

x = np.random.randint(0,5,(591,49))
X = pd.DataFrame.from_records(x)
y = pd.DataFrame.from_records(np.random.randint(0,1,(571,1)))
y2 = pd.DataFrame.from_records(np.random.randint(1,2,(20,1)))
y3 = pd.concat([y, y2])
y3['index'] = np.arange(len(y3))
y3 = y3.set_index('index')
y3 = y3.reindex(np.random.permutation(y3.index))
y3.columns = ['Outcome']
df = pd.concat([X, y3], axis = 1)
y = y3

In [4]:
#Instantiate algos and fit data
lr = LogisticRegression()
lr.fit(X,y)

rf = RandomForestClassifier()
rf.fit(X,y)

gb = XGBClassifier()
gb.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [5]:
#Find APPARENT predictions

#LR
preds_app_lr = lr.predict(X)
probs_app_lr = lr.predict_proba(X)[:,1]

#RF
preds_app_rf = rf.predict(X)
probs_app_rf = rf.predict_proba(X)[:,1]

#GB
preds_app_gb = gb.predict(X)
probs_app_gb = gb.predict_proba(X)[:,1]

In [6]:
#Determine APPARENT metrics

#LR
acc_app_lr = accuracy_score(y, preds_app_lr)
bacc_app_lr = balanced_accuracy_score(y, preds_app_lr)
roc_app_lr = roc_auc_score(y, probs_app_lr)
bs_app_lr = brier_score_loss(y, probs_app_lr)
prec_app_lr = precision_score(y, preds_app_lr)
rec_app_lr = recall_score(y, preds_app_lr)

prec_app_lr2, rec_app_lr2, _ = precision_recall_curve(y, probs_app_lr)
prc_app_lr = auc(rec_app_lr2, prec_app_lr2)

#RF
acc_app_rf = accuracy_score(y, preds_app_rf)
bacc_app_rf = balanced_accuracy_score(y, preds_app_rf)
roc_app_rf = roc_auc_score(y, probs_app_rf)
bs_app_rf = brier_score_loss(y, probs_app_rf)
prec_app_rf = precision_score(y, preds_app_rf)
rec_app_rf = recall_score(y, preds_app_rf)

prec_app_rf2, rec_app_rf2, _ = precision_recall_curve(y, probs_app_rf)
prc_app_rf = auc(rec_app_rf2, prec_app_rf2)

#GB
acc_app_gb = accuracy_score(y, preds_app_gb)
bacc_app_gb = balanced_accuracy_score(y, preds_app_gb)
roc_app_gb = roc_auc_score(y, probs_app_gb)
bs_app_gb = brier_score_loss(y, probs_app_gb)
prec_app_gb = precision_score(y, preds_app_gb)
rec_app_gb = recall_score(y, preds_app_gb)

prec_app_gb2, rec_app_gb2, _ = precision_recall_curve(y, probs_app_gb)
prc_app_gb = auc(rec_app_gb2, prec_app_gb2)


In [7]:
#Create arrays for optimism values

#LR
acc_lr = []
bacc_lr = []
roc_lr = []
bs_lr = []
prec_lr = []
rec_lr = []
prc_lr = []

#RF
acc_rf = []
bacc_rf = []
roc_rf = []
bs_rf = []
prec_rf = []
rec_rf = []
prc_rf = []

#GB
acc_gb = []
bacc_gb = []
roc_gb = []
bs_gb = []
prec_gb = []
rec_gb = []
prc_gb = []

In [8]:
for i in range(1, B):
    #Create bootstrap sample, fit algo, make predictions
    boot = resample(df, replace=True, n_samples=len(X)) #Create bootstrapped dataset
    y_b = boot.Outcome 
    X_b = boot.drop('Outcome', axis=1)


    lr.fit(X_b, y_b) #Fit LR using boostrapped data    
    pred_b_lr = lr.predict(X_b)
    prob_b_lr = lr.predict_proba(X_b)[:,1]
    pred_o_lr = lr.predict(X)
    prob_o_lr = lr.predict_proba(X)[:,1]

    rf.fit(X_b, y_b) #Fit rf using boostrapped data    
    pred_b_rf = rf.predict(X_b)
    prob_b_rf = rf.predict_proba(X_b)[:,1]
    pred_o_rf = rf.predict(X)
    prob_o_rf = rf.predict_proba(X)[:,1]

    gb.fit(X_b, y_b) #Fit gb using boostrapped data    
    pred_b_gb = gb.predict(X_b)
    prob_b_gb = gb.predict_proba(X_b)[:,1]
    pred_o_gb = gb.predict(X)
    prob_o_gb = gb.predict_proba(X)[:,1]
    
    
    #Accuracy
    acc_b_lr = accuracy_score(y_b, pred_b_lr)
    acc_o_lr = accuracy_score(y, pred_o_lr)
    acc_lr.append(acc_b_lr-acc_o_lr)
    
    acc_b_rf = accuracy_score(y_b, pred_b_rf)
    acc_o_rf = accuracy_score(y, pred_o_rf)
    acc_rf.append(acc_b_rf-acc_o_rf)
    
    acc_b_gb = accuracy_score(y_b, pred_b_gb)
    acc_o_gb = accuracy_score(y, pred_o_gb)
    acc_gb.append(acc_b_gb-acc_o_gb)    
    
    #Balanced accuracy
    bacc_b_lr = balanced_accuracy_score(y_b, pred_b_lr)
    bacc_o_lr = balanced_accuracy_score(y, pred_o_lr)
    bacc_lr.append(bacc_b_lr - bacc_o_lr)
    
    bacc_b_rf = balanced_accuracy_score(y_b, pred_b_rf)
    bacc_o_rf = balanced_accuracy_score(y, pred_o_rf)
    bacc_rf.append(bacc_b_rf - bacc_o_rf)
    
    bacc_b_gb = balanced_accuracy_score(y_b, pred_b_gb)
    bacc_o_gb = balanced_accuracy_score(y, pred_o_gb)
    bacc_gb.append(bacc_b_gb - bacc_o_gb)
    
    #AUROC
    roc_b_lr = roc_auc_score(y_b, prob_b_lr)
    roc_o_lr = roc_auc_score(y, prob_o_lr)
    roc_lr.append(roc_b_lr-roc_o_lr)
    
    roc_b_rf = roc_auc_score(y_b, prob_b_rf)
    roc_o_rf = roc_auc_score(y, prob_o_rf)
    roc_rf.append(roc_b_rf-roc_o_rf)    

    roc_b_gb = roc_auc_score(y_b, prob_b_gb)
    roc_o_gb = roc_auc_score(y, prob_o_gb)
    roc_gb.append(roc_b_gb-roc_o_gb)    
    
    #Brier
    bs_b_lr = brier_score_loss(y_b, prob_b_lr)
    bs_o_lr = brier_score_loss(y, prob_o_lr)
    bs_lr.append(bs_b_lr - bs_o_lr)

    bs_b_rf = brier_score_loss(y_b, prob_b_rf)
    bs_o_rf = brier_score_loss(y, prob_o_rf)
    bs_rf.append(bs_b_rf - bs_o_rf)    
    
    bs_b_gb = brier_score_loss(y_b, prob_b_gb)
    bs_o_gb = brier_score_loss(y, prob_o_gb)
    bs_gb.append(bs_b_gb - bs_o_gb)    
    
    #Precision
    prec_b_lr = precision_score(y_b, pred_b_lr)
    prec_o_lr = precision_score(y, pred_o_lr)
    prec_lr.append(prec_b_lr - prec_o_lr)

    prec_b_rf = precision_score(y_b, pred_b_rf)
    prec_o_rf = precision_score(y, pred_o_rf)
    prec_rf.append(prec_b_rf - prec_o_rf)    

    prec_b_gb = precision_score(y_b, pred_b_gb)
    prec_o_gb = precision_score(y, pred_o_gb)
    prec_gb.append(prec_b_gb - prec_o_gb)    
    
    #Recall
    rec_b_lr = recall_score(y_b, pred_b_lr)
    rec_o_lr = recall_score(y, pred_o_lr)
    rec_lr.append(rec_b_lr - rec_o_lr)

    rec_b_rf = recall_score(y_b, pred_b_rf)
    rec_o_rf = recall_score(y, pred_o_rf)
    rec_rf.append(rec_b_rf - rec_o_rf)    

    rec_b_gb = recall_score(y_b, pred_b_gb)
    rec_o_gb = recall_score(y, pred_o_gb)
    rec_gb.append(rec_b_gb - rec_o_gb)    
    
    #PRC
    prec_b_lr2, rec_b_lr2, _ = precision_recall_curve(y_b, prob_b_lr)
    prc_b_lr = auc(rec_b_lr2, prec_b_lr2)

    prec_o_lr2, rec_o_lr2, _ = precision_recall_curve(y, prob_o_lr)
    prc_o_lr = auc(rec_o_lr2, prec_o_lr2)
    
    prc_lr.append(prc_b_lr-prc_o_lr)
    
    prec_b_rf2, rec_b_rf2, _ = precision_recall_curve(y_b, prob_b_rf)
    prc_b_rf = auc(rec_b_rf2, prec_b_rf2)

    prec_o_rf2, rec_o_rf2, _ = precision_recall_curve(y, prob_o_rf)
    prc_o_rf = auc(rec_o_rf2, prec_o_rf2)
    
    prc_rf.append(prc_b_rf-prc_o_rf)    

    prec_b_gb2, rec_b_gb2, _ = precision_recall_curve(y_b, prob_b_gb)
    prc_b_gb = auc(rec_b_gb2, prec_b_gb2)

    prec_o_gb2, rec_o_gb2, _ = precision_recall_curve(y, prob_o_gb)
    prc_o_gb = auc(rec_o_gb2, prec_o_gb2)
    
    prc_gb.append(prc_b_gb-prc_o_gb)    

In [9]:
# Calculate optimisms and adjusted results

# Accuracy
acc_opt_lr = np.mean(acc_lr)
acc_adj_lr = acc_app_lr - acc_opt_lr

acc_opt_rf = np.mean(acc_rf)
acc_adj_rf = acc_app_rf - acc_opt_rf

acc_opt_gb = np.mean(acc_gb)
acc_adj_gb = acc_app_gb - acc_opt_gb

# Balanced accuracy
bacc_opt_lr = np.mean(bacc_lr)
bacc_adj_lr = bacc_app_lr - bacc_opt_lr

bacc_opt_rf = np.mean(bacc_rf)
bacc_adj_rf = bacc_app_rf - bacc_opt_rf

bacc_opt_gb = np.mean(bacc_gb)
bacc_adj_gb = bacc_app_gb - bacc_opt_gb

# AUROC
roc_opt_lr = np.mean(roc_lr)
roc_adj_lr = roc_app_lr - roc_opt_lr

roc_opt_rf = np.mean(roc_rf)
roc_adj_rf = roc_app_rf - roc_opt_rf

roc_opt_gb = np.mean(roc_gb)
roc_adj_gb = roc_app_gb - roc_opt_gb

# Precision
prec_opt_lr = np.mean(prec_lr)
prec_adj_lr = prec_app_lr - prec_opt_lr

prec_opt_rf = np.mean(prec_rf)
prec_adj_rf = prec_app_rf - prec_opt_rf

prec_opt_gb = np.mean(prec_gb)
prec_adj_gb = prec_app_gb - prec_opt_gb

# Recall
rec_opt_lr = np.mean(rec_lr)
rec_adj_lr = rec_app_lr - rec_opt_lr

rec_opt_rf = np.mean(rec_rf)
rec_adj_rf = rec_app_rf - rec_opt_rf

rec_opt_gb = np.mean(rec_gb)
rec_adj_gb = rec_app_gb - rec_opt_gb

# Brier score
bs_opt_lr = np.mean(bs_lr)
bs_adj_lr = bs_app_lr - bs_opt_lr

bs_opt_rf = np.mean(bs_rf)
bs_adj_rf = bs_app_rf - bs_opt_rf

bs_opt_gb = np.mean(bs_gb)
bs_adj_gb = bs_app_gb - bs_opt_gb

# AUPRC
prc_opt_lr = np.mean(prc_lr)
prc_adj_lr = prc_app_lr - prc_opt_lr

prc_opt_rf = np.mean(prc_rf)
prc_adj_rf = prc_app_rf - prc_opt_rf

prc_opt_gb = np.mean(prc_gb)
prc_adj_gb = prc_app_gb - prc_opt_gb

print('LR')
print('Adjusted AUROC', roc_adj_lr)#, '; Optimism', roc_opt_lr)
print('Adjusted accuracy', acc_adj_lr)#, '; Optimism', acc_opt_lr)
print('Adjusted balanced accuracy', bacc_adj_lr)#, '; Optimism', bacc_opt_lr)
print('Adjusted precision', prec_adj_lr)#, '; Optimism', prec_opt_lr)
print('Adjusted recall', rec_adj_lr)#, '; Optimism', rec_opt_lr)
print('Adjusted AUPRC', prc_adj_lr)#, '; Optimism', prc_opt_lr)
print('Adjusted Brier score', bs_adj_lr)#, '; Optimism', bs_opt_lr)
print('\n')

print('RF')
print('Adjusted AUROC', roc_adj_rf)#, '; Optimism', roc_opt_rf)
print('Adjusted accuracy', acc_adj_rf)#, '; Optimism', acc_opt_rf)
print('Adjusted balanced accuracy', bacc_adj_rf)#, '; Optimism', bacc_opt_rf)
print('Adjusted precision', prec_adj_rf)#, '; Optimism', prec_opt_rf)
print('Adjusted recall', rec_adj_rf)#, '; Optimism', rec_opt_rf)
print('Adjusted AUPRC', prc_adj_rf)#, '; Optimism', prc_opt_rf)
print('Adjusted Brier score', bs_adj_rf)#, '; Optimism', bs_opt_rf)
print('\n')

print('GB')
print('Adjusted AUROC', roc_adj_gb)#, '; Optimism', roc_opt_gb)
print('Adjusted accuracy', acc_adj_gb)#, '; Optimism', acc_opt_gb)
print('Adjusted balanced accuracy', bacc_adj_gb)#, '; Optimism', bacc_opt_gb)
print('Adjusted precision', prec_adj_gb)#, '; Optimism', prec_opt_gb)
print('Adjusted recall', rec_adj_gb)#, '; Optimism', rec_opt_gb)
print('Adjusted AUPRC', prc_adj_gb)#, '; Optimism', prc_opt_gb)
print('Adjusted Brier score', bs_adj_gb)#, '; Optimism', bs_opt_gb)

LR
Adjusted AUROC 0.8290024712862887
Adjusted accuracy 0.9618341793570221
Adjusted balanced accuracy 0.5705317309100856
Adjusted precision 0.6092913758344725
Adjusted recall 0.15082820188203602
Adjusted AUPRC 0.1258640666555646
Adjusted Brier score 0.03526684444561276


RF
Adjusted AUROC 0.921333887915937
Adjusted accuracy 0.9873874788494078
Adjusted balanced accuracy 0.8152017663151006
Adjusted precision 0.9955719738208129
Adjusted recall 0.6305156166932482
Adjusted AUPRC 0.7382087192478024
Adjusted Brier score 0.017484746531302876


GB
Adjusted AUROC 0.8957535026269703
Adjusted accuracy 0.9857157360406091
Adjusted balanced accuracy 0.8165965504563338
Adjusted precision 0.9264248074337084
Adjusted recall 0.635168582523876
Adjusted AUPRC 0.7161010302112542
Adjusted Brier score 0.013890580195884435
